# Lezzetli Asya ve Hint Mutfağı: Bir sınıflandırma modeli oluşturun


## Mutfak Sınıflandırıcıları 2

Bu ikinci sınıflandırma dersinde, kategorik verileri sınıflandırmanın `daha fazla yolunu` keşfedeceğiz. Ayrıca bir sınıflandırıcıyı diğerine tercih etmenin sonuçlarını öğreneceğiz.

### [**Ders Öncesi Testi**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Ön Koşul**

Önceki dersleri tamamladığınızı varsayıyoruz, çünkü daha önce öğrendiğimiz bazı kavramları burada devam ettireceğiz.

Bu ders için aşağıdaki paketlere ihtiyacımız olacak:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/), veri bilimini daha hızlı, kolay ve eğlenceli hale getirmek için tasarlanmış bir [R paketleri koleksiyonudur](https://www.tidyverse.org/packages).

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) çerçevesi, modelleme ve makine öğrenimi için bir [paketler koleksiyonudur](https://www.tidymodels.org/packages/).

-   `themis`: [themis paketi](https://themis.tidymodels.org/), Dengesiz Verilerle Çalışmak için Ek Tarif Adımları sağlar.

Bu paketleri şu şekilde yükleyebilirsiniz:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternatif olarak, aşağıdaki script, bu modülü tamamlamak için gerekli paketlere sahip olup olmadığınızı kontrol eder ve eksik olanları sizin için yükler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Bir sınıflandırma haritası**

[Önceki dersimizde](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), şu soruyu ele almaya çalıştık: birden fazla model arasında nasıl seçim yaparız? Büyük ölçüde, bu seçim veri özelliklerine ve çözmek istediğimiz problemin türüne (örneğin sınıflandırma veya regresyon) bağlıdır.

Daha önce, Microsoft'un cheat sheet'ini kullanarak verileri sınıflandırırken sahip olduğunuz çeşitli seçenekleri öğrenmiştik. Python'un Makine Öğrenimi çerçevesi olan Scikit-learn, benzer ancak daha ayrıntılı bir cheat sheet sunar ve bu, tahmincilerinizi (sınıflandırıcılar için başka bir terim) daha da daraltmanıza yardımcı olabilir:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> İpucu: [bu haritayı çevrimiçi ziyaret edin](https://scikit-learn.org/stable/tutorial/machine_learning_map/) ve belgeleri okumak için yol boyunca tıklayın.
>
> [Tidymodels referans sitesi](https://www.tidymodels.org/find/parsnip/#models) ayrıca farklı model türleri hakkında mükemmel bir dokümantasyon sunar.

### **Plan** 🗺️

Bu harita, verilerinizi net bir şekilde anladığınızda çok faydalıdır, çünkü yolları boyunca 'yürüyerek' bir karara varabilirsiniz:

-   \>50 örneğimiz var

-   Bir kategori tahmin etmek istiyoruz

-   Etiketlenmiş verilerimiz var

-   100K'dan az örneğimiz var

-   ✨ Linear SVC seçebiliriz

-   Eğer bu işe yaramazsa, çünkü sayısal verilerimiz var

    -   ✨ KNeighbors Classifier deneyebiliriz

        -   Eğer bu da işe yaramazsa, ✨ SVC ve ✨ Ensemble Classifiers deneyin

Bu takip edilmesi çok faydalı bir yol. Şimdi, [tidymodels](https://www.tidymodels.org/) modelleme çerçevesini kullanarak işe koyulalım: iyi istatistiksel uygulamaları teşvik etmek için geliştirilmiş, tutarlı ve esnek bir R paketleri koleksiyonu 😊.

## 2. Veriyi böl ve dengesiz veri setiyle başa çık.

Önceki derslerimizden, mutfaklarımız arasında ortak olan bir dizi bileşen olduğunu öğrendik. Ayrıca, mutfakların sayısında oldukça eşitsiz bir dağılım vardı.

Bunlarla şu şekilde başa çıkacağız:

-   Farklı mutfaklar arasında kafa karışıklığı yaratan en yaygın bileşenleri `dplyr::select()` kullanarak çıkaracağız.

-   Veriyi modellemeye hazırlamak için bir `recipe` kullanarak bir `over-sampling` algoritması uygulayacağız.

Yukarıdakilere önceki derste zaten baktık, bu yüzden bu iş kolay olacak 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Dengesiz Verilerle Başa Çıkma

Dengesiz veriler genellikle model performansı üzerinde olumsuz etkilere sahiptir. Birçok model, gözlem sayısının eşit olduğu durumlarda en iyi performansı gösterir ve bu nedenle dengesiz verilerle başa çıkmakta zorlanır.

Dengesiz veri setleriyle başa çıkmanın temel olarak iki yolu vardır:

-   azınlık sınıfına gözlem eklemek: `Over-sampling` (Aşırı örnekleme), örneğin, bir SMOTE algoritması kullanarak. Bu algoritma, azınlık sınıfının yeni örneklerini, bu vakaların en yakın komşularını kullanarak sentetik olarak üretir.

-   çoğunluk sınıfından gözlem çıkarmak: `Under-sampling` (Azaltılmış örnekleme)

Önceki dersimizde, dengesiz veri setleriyle bir `recipe` (tarif) kullanarak nasıl başa çıkılacağını göstermiştik. Bir tarif, bir veri setine hangi adımların uygulanması gerektiğini tanımlayan bir plan olarak düşünülebilir. Bizim durumumuzda, `training set` (eğitim seti) için mutfak türlerimizin sayısında eşit bir dağılım elde etmek istiyoruz. Hadi başlayalım.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Artık modelleri eğitmeye hazırız 👩‍💻👨‍💻!

## 3. Multinom regresyon modellerinin ötesinde

Önceki dersimizde multinom regresyon modellerine baktık. Şimdi sınıflandırma için daha esnek modelleri keşfedelim.

### Destek Vektör Makineleri

Sınıflandırma bağlamında, `Destek Vektör Makineleri` sınıfları "en iyi" şekilde ayıran bir *hiper düzlem* bulmaya çalışan bir makine öğrenimi tekniğidir. Basit bir örneğe bakalım:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ sınıfları ayırmaz. H2~ ayırır, ancak yalnızca küçük bir boşlukla. H3~ ise sınıfları maksimum boşlukla ayırır.

#### Doğrusal Destek Vektör Sınıflandırıcı

Destek-Vektör kümeleme (SVC), ML teknikleri ailesinden Destek-Vektör makinelerinin bir alt dalıdır. SVC'de, hiper düzlem, eğitim gözlemlerinin `çoğunu` doğru bir şekilde ayıracak şekilde seçilir, ancak bazı gözlemleri `yanlış sınıflandırabilir`. Bazı noktaların yanlış tarafta olmasına izin vererek, SVM aykırı değerlere karşı daha dayanıklı hale gelir ve dolayısıyla yeni verilere daha iyi genelleme yapar. Bu ihlali düzenleyen parametre `maliyet` olarak adlandırılır ve varsayılan değeri 1'dir (bkz. `help("svm_poly")`).

Bir polinom SVM modelinde `degree = 1` ayarlayarak doğrusal bir SVC oluşturalım.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Artık ön işleme adımlarını ve model spesifikasyonunu bir *iş akışına* dahil ettiğimize göre, linear SVC'yi eğitip sonuçları değerlendirerek devam edebiliriz. Performans metrikleri için, `accuracy`, `sensitivity`, `Positive Predicted Value` ve `F Measure` değerlendirecek bir metrik seti oluşturalım.

> `augment()` verilen verilere tahminler için sütun(lar) ekleyecektir.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Destek Vektör Makinesi

Destek vektör makinesi (SVM), sınıflar arasındaki doğrusal olmayan bir sınırı karşılamak için destek vektör sınıflandırıcısının bir uzantısıdır. Temelde, SVM'ler sınıflar arasındaki doğrusal olmayan ilişkileri uyarlamak için özellik uzayını genişletmek amacıyla *çekirdek hilesi*ni kullanır. SVM'ler tarafından kullanılan popüler ve son derece esnek bir çekirdek fonksiyonu *Radyal tabanlı fonksiyon*dur. Şimdi, verilerimiz üzerinde nasıl bir performans göstereceğini görelim.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Çok daha iyi 🤩!

> ✅ Lütfen bakınız:
>
> -   [*Destek Vektör Makineleri*](https://bradleyboehmke.github.io/HOML/svm.html), R ile Uygulamalı Makine Öğrenimi
>
> -   [*Destek Vektör Makineleri*](https://www.statlearning.com/), R ile İstatistiksel Öğrenime Giriş
>
> daha fazla bilgi için.

### En Yakın Komşu Sınıflandırıcılar

*K*-en yakın komşu (KNN), her bir gözlemin diğer gözlemlere olan *benzerliğine* dayanarak tahmin edildiği bir algoritmadır.

Haydi, verilerimize bir tane uygulayalım.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bu modelin performansı pek iyi görünmüyor. Muhtemelen modelin argümanlarını değiştirmek (bkz. `help("nearest_neighbor")`) model performansını artıracaktır. Mutlaka denemelisiniz.

> ✅ Lütfen bakınız:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> *K*-En Yakın Komşu sınıflandırıcıları hakkında daha fazla bilgi edinmek için.

### Topluluk sınıflandırıcıları

Topluluk algoritmaları, birden fazla temel tahmin ediciyi birleştirerek optimal bir model oluşturur. Bu, şu yöntemlerle yapılır:

`bagging`: temel modellerin bir koleksiyonuna *ortalama alma fonksiyonu* uygulamak

`boosting`: birbiri üzerine inşa edilen bir model dizisi oluşturarak tahmin performansını iyileştirmek.

Hadi bir Random Forest modelini deneyerek başlayalım. Bu model, büyük bir karar ağacı koleksiyonu oluşturur ve ardından daha iyi bir genel model için bir ortalama alma fonksiyonu uygular.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Tebrikler 👏!

Hadi bir de Boosted Tree modeliyle deney yapalım.

Boosted Tree, bir dizi ardışık karar ağacı oluşturan ve her ağacın önceki ağaçların sonuçlarına bağlı olduğu bir topluluk yöntemini tanımlar. Amaç, hatayı kademeli olarak azaltmaktır. Bu yöntem, yanlış sınıflandırılmış öğelerin ağırlıklarına odaklanır ve bir sonraki sınıflandırıcıyı düzeltmek için uyumu ayarlar.

Bu modeli oluşturmanın farklı yolları vardır (bkz. `help("boost_tree")`). Bu örnekte, Boosted Tree'leri `xgboost` motoru aracılığıyla oluşturacağız.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Lütfen bakınız:
>
> -   [Sosyal Bilimciler için Makine Öğrenimi](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [R ile Uygulamalı Makine Öğrenimi](https://bradleyboehmke.github.io/HOML/)
>
> -   [R Uygulamalarıyla İstatistiksel Öğrenime Giriş](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - xgboost'a iyi bir alternatif olan AdaBoost modelini inceler.
>
> Ensemble sınıflandırıcılar hakkında daha fazla bilgi edinmek için.

## 4. Ekstra - birden fazla modeli karşılaştırma

Bu laboratuvarda oldukça fazla model oluşturduk 🙌. Farklı ön işleme setlerinden ve/veya model tanımlarından birçok iş akışı oluşturmak ve ardından performans metriklerini tek tek hesaplamak yorucu veya zahmetli olabilir.

Bunu, eğitim seti üzerinde bir iş akışı listesini eğiten ve ardından test setine dayalı performans metriklerini döndüren bir fonksiyon oluşturarak çözebilir miyiz, bir bakalım. Liste içindeki her bir elemana fonksiyon uygulamak için [purrr](https://purrr.tidyverse.org/) paketinden `map()` ve `map_dfr()` fonksiyonlarını kullanacağız.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) fonksiyonları, birçok for döngüsünü daha özlü ve okunması daha kolay bir kodla değiştirmenize olanak tanır. [`map()`](https://purrr.tidyverse.org/reference/map.html) fonksiyonları hakkında bilgi edinmek için en iyi yer, R for Data Science kitabındaki [iterasyon bölümü](http://r4ds.had.co.nz/iteration.html)dir.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) paketi, kullanıcıların çok sayıda modeli oluşturmasını ve kolayca eğitmesini sağlar, ancak esas olarak `çapraz doğrulama` gibi yeniden örnekleme teknikleriyle çalışmak üzere tasarlanmıştır. Bu, henüz ele almadığımız bir yaklaşımdır.

## **🚀Meydan Okuma**

Bu tekniklerin her birinin, örneğin SVM'lerdeki `cost`, KNN'deki `neighbors`, Random Forest'taki `mtry` (Rastgele Seçilen Tahminciler) gibi ayarlayabileceğiniz çok sayıda parametresi vardır.

Her birinin varsayılan parametrelerini araştırın ve bu parametreleri değiştirmenin modelin kalitesi açısından ne anlama gelebileceğini düşünün.

Belirli bir model ve parametreleri hakkında daha fazla bilgi edinmek için şu komutu kullanabilirsiniz: `help("model")` örneğin `help("rand_forest")`.

> Uygulamada, genellikle bu parametrelerin *en iyi değerlerini* tahmin etmek için bir `simüle edilmiş veri seti` üzerinde birçok model eğitir ve bu modellerin ne kadar iyi performans gösterdiğini ölçeriz. Bu sürece **tuning** (ince ayar) denir.

### [**Ders Sonrası Testi**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Gözden Geçirme ve Kendi Kendine Çalışma**

Bu derslerde çok fazla teknik terim var, bu yüzden [bu listeyi](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) gözden geçirmek için bir dakikanızı ayırın. Faydalı terminolojiler içeriyor!

#### TEŞEKKÜRLER:

[`Allison Horst`](https://twitter.com/allison_horst/) R'ı daha sıcak ve ilgi çekici hale getiren harika illüstrasyonlar oluşturduğu için. Daha fazla illüstrasyonu [galerisinde](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) bulabilirsiniz.

[Cassie Breviu](https://www.twitter.com/cassieview) ve [Jen Looper](https://www.twitter.com/jenlooper) bu modülün orijinal Python versiyonunu oluşturdukları için ♥️

Keyifli Öğrenmeler,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Öğrenci Elçisi.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>@allison_horst tarafından yapılmış bir çalışma</figcaption>



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalardan sorumlu değiliz.
